In [2]:
import os
import re
import scipy
from scipy import stats
import pickle
import subprocess
import shlex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
merged_df = pd.read_csv('merged_train.csv')

merged_test_df = pd.read_csv('merged_test.csv')

In [19]:
merged_holdout_df = pd.read_csv('merged_holdout.csv')

In [20]:
print(merged_holdout_df.shape)
merged_holdout_df.head()

(9838, 70)


,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour
0,20017191,2032-12-04 04:56:00,31,16.0,35,9.2,99,1.6,156,140.0,...,0,1,0,0,0,0,0,0,0,11.233333
1,20017191,2032-12-04 14:19:00,18,23.0,40,8.4,105,1.5,136,141.0,...,0,1,0,0,0,0,0,0,0,20.616667
2,20036035,2022-10-05 23:22:00,29,16.0,19,9.1,102,0.8,486,143.0,...,0,0,0,0,0,0,0,0,1,1.433333
3,20036035,2022-10-06 03:36:00,31,13.0,18,9.1,104,1.0,386,144.0,...,0,0,0,0,0,0,0,0,1,5.666667
4,20037205,2022-03-05 21:55:00,17,21.0,11,8.7,104,0.6,129,138.0,...,0,0,1,0,0,0,0,0,0,2.166667


In [21]:
merged_holdout_df.id.nunique()

2042

In [5]:
grouped_df = merged_df.groupby('id')

In [6]:
grouped_df.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour
0,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142.0,...,0,0,0,1,0,0,0,0,0,5.350000
1,20001305,1978-03-25 13:45:00,13,25.0,48,10.8,107,0.9,149,140.0,...,0,0,0,1,0,0,0,0,0,10.766667
2,20001305,1978-03-25 21:55:00,13,24.0,50,10.8,108,0.9,131,141.0,...,0,0,0,1,0,0,0,0,0,18.933333
3,20001361,2043-05-04 17:24:00,14,22.0,28,6.3,107,2.5,161,137.0,...,0,0,0,1,0,0,0,0,0,0.533333
4,20001361,2043-05-04 21:07:00,15,20.0,32,6.5,108,2.5,124,137.0,...,0,0,0,1,0,0,0,0,0,4.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64980,29998399,2012-05-09 17:50:00,10,23.0,12,9.2,107,1.1,129,140.0,...,0,0,0,0,0,0,0,0,1,20.416667
64981,29998399,2012-05-09 19:45:00,11,24.0,12,8.2,105,1.1,122,140.0,...,0,0,0,0,0,0,0,0,1,22.333333
64982,29999625,2057-11-07 13:33:00,17,21.0,14,8.5,104,1.2,109,142.0,...,0,0,0,0,0,0,1,0,0,1.700000
64983,29999625,2057-11-08 01:36:00,15,24.0,21,8.6,110,1.6,122,149.0,...,0,0,0,0,0,0,1,0,0,13.750000


In [7]:
# Assuming 'df' is your DataFrame with time series data and 'id' is the column you want to group by
# Example DataFrame structure:
df = pd.DataFrame({'id': [1, 1, 2, 2, 3], 'timestamp': [pd.Timestamp('2024-01-01'), pd.Timestamp('2024-01-02'), pd.Timestamp('2024-01-01'), pd.Timestamp('2024-01-02'), pd.Timestamp('2024-01-01')], 'value': [10, 20, 30, 40, 50]})

# Group the DataFrame by 'id'
grouped = df.groupby('id')

# Now, 'grouped' is a DataFrameGroupBy object, which contains groups of dataframes for each unique value in the 'id' column.

# You can iterate over the groups, or access specific groups using the get_group method, for example:
for group_id, group_data in grouped:
    print(f"Group ID: {group_id}")
    print(group_data)

Group ID: 1
   id  timestamp  value
0   1 2024-01-01     10
1   1 2024-01-02     20
Group ID: 2
   id  timestamp  value
2   2 2024-01-01     30
3   2 2024-01-02     40
Group ID: 3
   id  timestamp  value
4   3 2024-01-01     50


In [8]:
grouped.head()

,id,timestamp,value
0,1,2024-01-01,10
1,1,2024-01-02,20
2,2,2024-01-01,30
3,2,2024-01-02,40
4,3,2024-01-01,50


In [23]:
# id_list = merged_df.id.unique().tolist()

# test_id_list = merged_test_df.id.unique().tolist()

holdout_id_list = merged_holdout_df.id.unique().tolist()

In [24]:
len(holdout_id_list)

2042

In [25]:
# id_list.sort()

# test_id_list.sort()

holdout_id_list.sort()

In [70]:
# counter = 0
# for id in id_list:
#     id_df = merged_df[merged_df['id'] == id]
#     if id_df.shape[0] >= 5:
#         counter += 1
# print(counter)

4146


In [61]:
# trial_df = merged_df[merged_df['id'] == 20001305]

# trial_df.head()
# result_df = pd.DataFrame()
# for index, row in trial_df.iterrows():
#     #print(row)
#     row_dict = row.to_dict()
#     result_df = result_df.append(row_dict, ignore_index=True)
# result_df.shape

(3, 70)

In [26]:
def get_time_step(df, id_list, steps):
    result_df = pd.DataFrame()
    for id in id_list:
        id_df = df[df['id'] == id]
        if id_df.shape[0] >= steps:
            id_df = id_df.sort_values(by='hour', ascending=False)
            counter = 0
            for index, row in id_df.iterrows():
                if counter < steps:
                    row_dict = row.to_dict()
                    result_df = result_df.append(row_dict, ignore_index=True)
                    counter += 1
        else:
            id_df = id_df.sort_values(by='hour', ascending=False)
            n = steps - id_df.shape[0]
            for index, row in id_df.iterrows():
                row_dict = row.to_dict()
                result_df = result_df.append(row_dict, ignore_index=True)
            for i in range(n):
                result_df = result_df.append(row_dict, ignore_index=True)
            
    return result_df

In [27]:
holdout_df = get_time_step(merged_holdout_df, holdout_id_list, 5)

In [28]:
holdout_df.shape

(10210, 70)

In [79]:
id_df = result_df[result_df['id'] == 20008724]
id_df.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour
35,20008724.0,2024-02-27 01:53:00,12.0,25.0,14.0,9.0,103.0,0.5,112.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.05
36,20008724.0,2024-02-27 01:53:00,12.0,25.0,14.0,9.0,103.0,0.5,112.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.05
37,20008724.0,2024-02-27 01:53:00,12.0,25.0,14.0,9.0,103.0,0.5,112.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.05
38,20008724.0,2024-02-27 01:53:00,12.0,25.0,14.0,9.0,103.0,0.5,112.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.05
39,20008724.0,2024-02-27 01:53:00,12.0,25.0,14.0,9.0,103.0,0.5,112.0,137.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.05


In [3]:
# Start here - load processed timestep5 data

train_df = pd.read_excel('Timestep5_train.xlsx')
test_df = pd.read_excel('Timestep5_test.xlsx')
holdout_df = pd.read_excel('Timestep5_holdout.xlsx')

In [5]:
print(train_df.shape)
print(test_df.shape)
print(holdout_df.shape)

(71445, 70)
(20415, 70)
(10210, 70)


In [4]:
train_df.head(10)

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour,icu_cat
0,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,1
1,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,1
2,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,1
3,20001305,1978-03-25 13:45:00,13,25.0,48,10.8,107,0.9,149,140,...,0,0,1,0,0,0,0,0,10.766667,1
4,20001305,1978-03-25 21:55:00,13,24.0,50,10.8,108,0.9,131,141,...,0,0,1,0,0,0,0,0,18.933333,1
5,20001361,2043-05-04 17:24:00,14,22.0,28,6.3,107,2.5,161,137,...,0,0,1,0,0,0,0,0,0.533333,2
6,20001361,2043-05-04 21:07:00,15,20.0,32,6.5,108,2.5,124,137,...,0,0,1,0,0,0,0,0,4.250000,2
7,20001361,2043-05-05 04:27:00,15,23.0,36,7.2,108,2.9,98,142,...,0,0,1,0,0,0,0,0,11.583333,2
8,20001361,2043-05-05 11:50:00,15,23.0,27,8.4,107,1.3,134,142,...,0,0,1,0,0,0,0,0,18.966667,2
9,20001361,2043-05-05 15:02:00,18,23.0,40,7.9,107,3.7,123,144,...,0,0,1,0,0,0,0,0,22.166667,2


In [ ]:
# # Converting los_icu into 3 categorical bins

# low_thres = 2.5
# medium_thres = 5

# result_df['icu_cat'] = 10
# result_df.loc[result_df['los_icu'] < low_thres, 'icu_cat'] = 0
# result_df.loc[((result_df['los_icu'] >= low_thres) & (result_df['los_icu'] < medium_thres)), 'icu_cat'] = 1
# result_df.loc[result_df['los_icu'] >= medium_thres, 'icu_cat'] = 2

# # test_df['icu_cat'] = 10
# # test_df.loc[test_df['los_icu'] < low_thres, 'icu_cat'] = 0
# # test_df.loc[((test_df['los_icu'] >= low_thres) & (test_df['los_icu'] < medium_thres)), 'icu_cat'] = 1
# # test_df.loc[test_df['los_icu'] >= medium_thres, 'icu_cat'] = 2

# holdout_df['icu_cat'] = 10
# holdout_df.loc[holdout_df['los_icu'] < low_thres, 'icu_cat'] = 0
# holdout_df.loc[((holdout_df['los_icu'] >= low_thres) & (holdout_df['los_icu'] < medium_thres)), 'icu_cat'] = 1
# holdout_df.loc[holdout_df['los_icu'] >= medium_thres, 'icu_cat'] = 2

In [6]:
# Converting los_icu into 2 categorical bins

low_thres = 7
medium_thres = None

train_df['icu_cat'] = 10
train_df.loc[train_df['los_icu'] <= low_thres, 'icu_cat'] = 0
train_df.loc[train_df['los_icu'] > low_thres, 'icu_cat'] = 1

test_df['icu_cat'] = 10
test_df.loc[test_df['los_icu'] <= low_thres, 'icu_cat'] = 0
test_df.loc[test_df['los_icu'] > low_thres, 'icu_cat'] = 1

holdout_df['icu_cat'] = 10
holdout_df.loc[holdout_df['los_icu'] <= low_thres, 'icu_cat'] = 0
holdout_df.loc[holdout_df['los_icu'] > low_thres, 'icu_cat'] = 1

In [7]:
print(train_df.shape)
print(test_df.shape)
print(holdout_df.shape)
train_df.head(10)

(71445, 70)
(20415, 70)
(10210, 70)


,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour,icu_cat
0,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,0
1,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,0
2,20001305,1978-03-25 08:20:00,15,23.0,47,11.4,108,0.8,154,142,...,0,0,1,0,0,0,0,0,5.350000,0
3,20001305,1978-03-25 13:45:00,13,25.0,48,10.8,107,0.9,149,140,...,0,0,1,0,0,0,0,0,10.766667,0
4,20001305,1978-03-25 21:55:00,13,24.0,50,10.8,108,0.9,131,141,...,0,0,1,0,0,0,0,0,18.933333,0
5,20001361,2043-05-04 17:24:00,14,22.0,28,6.3,107,2.5,161,137,...,0,0,1,0,0,0,0,0,0.533333,0
6,20001361,2043-05-04 21:07:00,15,20.0,32,6.5,108,2.5,124,137,...,0,0,1,0,0,0,0,0,4.250000,0
7,20001361,2043-05-05 04:27:00,15,23.0,36,7.2,108,2.9,98,142,...,0,0,1,0,0,0,0,0,11.583333,0
8,20001361,2043-05-05 11:50:00,15,23.0,27,8.4,107,1.3,134,142,...,0,0,1,0,0,0,0,0,18.966667,0
9,20001361,2043-05-05 15:02:00,18,23.0,40,7.9,107,3.7,123,144,...,0,0,1,0,0,0,0,0,22.166667,0


In [ ]:
# holdout_df = holdout_df.drop(columns=['icu_outcome'])

In [12]:
# print(result_df.shape)
# print(test_df.shape)
# print(holdout_df.shape)

In [39]:
# # sorting hours to ascending order

# train_df = train_df.sort_values(by=['id', 'hour'], ascending=[True, True])
# test_df = test_df.sort_values(by=['id', 'hour'], ascending=[True, True])
# holdout_df = holdout_df.sort_values(by=['id', 'hour'], ascending=[True, True])

# train_df.reset_index(drop=True, inplace=True)
# test_df.reset_index(drop=True, inplace=True)
# holdout_df.reset_index(drop=True, inplace=True)

# test_df.head(20)

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,...,first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),hour,icu_cat
0,20001770,2017-01-26 03:56:00,14,22.0,23,7.4,86,1.3,80,122.0,...,0,0,0,0,0,0,1,0,7.150000,1
1,20001770,2017-01-26 03:56:00,14,22.0,23,7.4,86,1.3,80,122.0,...,0,0,0,0,0,0,1,0,7.150000,1
2,20001770,2017-01-26 03:56:00,14,22.0,23,7.4,86,1.3,80,122.0,...,0,0,0,0,0,0,1,0,7.150000,1
3,20001770,2017-01-26 03:56:00,14,22.0,23,7.4,86,1.3,80,122.0,...,0,0,0,0,0,0,1,0,7.150000,1
4,20001770,2017-01-26 03:56:00,14,22.0,23,7.4,86,1.3,80,122.0,...,0,0,0,0,0,0,1,0,7.150000,1
5,20004357,2057-08-05 16:37:00,21,24.0,24,9.9,101,1.1,177,142.0,...,0,1,0,0,0,0,0,0,4.250000,1
6,20004357,2057-08-05 16:37:00,21,24.0,24,9.9,101,1.1,177,142.0,...,0,1,0,0,0,0,0,0,4.250000,1
7,20004357,2057-08-05 16:37:00,21,24.0,24,9.9,101,1.1,177,142.0,...,0,1,0,0,0,0,0,0,4.250000,1
8,20004357,2057-08-05 16:37:00,21,24.0,24,9.9,101,1.1,177,142.0,...,0,1,0,0,0,0,0,0,4.250000,1
9,20004357,2057-08-06 04:35:00,12,29.0,29,9.2,105,1.1,155,142.0,...,0,1,0,0,0,0,0,0,16.216667,1


In [41]:
# # Can also Start here - load processed timestep5 data

# train_df.to_excel('Timestep5_train.xlsx', index=False)
# test_df.to_excel('Timestep5_test.xlsx', index=False)
# holdout_df.to_excel('Timestep5_holdout.xlsx', index=False)

In [8]:
columns_to_drop = ['id',
                   'charttime',
                   'hour',
                   'icu_cat',
                   'hosp_admittime',
                   'hosp_dischtime',
                   'icu_intime',
                   'icu_outtime',
                   'los_icu',
                   'icu_death',
                   'race'
                  ]

X_train_df = train_df.drop(columns=columns_to_drop)
X_test_df = test_df.drop(columns=columns_to_drop)
X_holdout_df = holdout_df.drop(columns=columns_to_drop)

print(X_train_df.shape)
print(X_test_df.shape)
                   


(71445, 59)
(20415, 59)


In [9]:
# Perform normalization using data from X_train to transform X_test

from sklearn.preprocessing import StandardScaler

num_cols = X_train_df.columns[X_train_df.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
# print(num_cols)
scaler = StandardScaler()
X_train_df[num_cols] = scaler.fit_transform(X_train_df[num_cols])
X_test_df[num_cols] = scaler.transform(X_test_df[num_cols])
X_holdout_df[num_cols] = scaler.transform(X_holdout_df[num_cols])

In [36]:
# X_df = X_df.drop(columns=['race'])
X_train_df.dtypes
X_train_df.columns.tolist()

['aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium',
 'hematocrit',
 'hemoglobin',
 'mch',
 'mchc',
 'mcv',
 'platelet',
 'rbc',
 'rdw',
 'wbc',
 'inr',
 'pt',
 'ptt',
 'gender',
 'admission_age',
 'weight_admit',
 'height',
 'charlson_score',
 'atrial_fibrillation',
 'malignant_cancer',
 'chf',
 'ckd',
 'cld',
 'copd',
 'diabetes',
 'hypertension',
 'ihd',
 'stroke',
 'race_encode_African',
 'race_encode_Asian',
 'race_encode_Caucasian',
 'race_encode_Hispanic',
 'race_encode_Not Specified',
 'race_encode_South American',
 'admission_type_DIRECT EMER.',
 'admission_type_DIRECT OBSERVATION',
 'admission_type_ELECTIVE',
 'admission_type_EU OBSERVATION',
 'admission_type_EW EMER.',
 'admission_type_OBSERVATION ADMIT',
 'admission_type_SURGICAL SAME DAY ADMISSION',
 'admission_type_URGENT',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'first_careunit_Coronary Care Unit (CCU)',
 'first_careunit_Medical Intensi

In [11]:

# steps = 5

# numpy_data = X_df.values

# print(len(numpy_data))
# time_input = []
# labels = []

# for i in range(int(len(numpy_data)/steps)):
#     sample = X_df.iloc[i*steps:i*steps+steps]
#     label = result_df.iloc[i*steps][-1]
#     time_input.append(sample)
#     labels.append(label)

# time_input = np.array(time_input)
# labels = np.array(labels)

In [12]:
# Converting training, testing and holdout data into 3D numpy array

steps = 5

numpy_train_data = X_train_df.values
numpy_test_data = X_test_df.values
numpy_holdout_data = X_holdout_df.values

# print(len(numpy_data))
X_train_input = []
y_train = []
X_test_input = []
y_test = []
X_holdout_input = []
y_holdout = []


for i in range(int(len(numpy_train_data)/steps)):
    sample = X_train_df.iloc[i*steps:i*steps+steps]
    label = train_df.iloc[i*steps][-1]
    X_train_input.append(sample)
    y_train.append(label)

for i in range(int(len(numpy_test_data)/steps)):
    sample = X_test_df.iloc[i*steps:i*steps+steps]
    label = test_df.iloc[i*steps][-1]
    X_test_input.append(sample)
    y_test.append(label)

for i in range(int(len(numpy_holdout_data)/steps)):
    sample = X_holdout_df.iloc[i*steps:i*steps+steps]
    label = holdout_df.iloc[i*steps][-1]
    X_holdout_input.append(sample)
    y_holdout.append(label)

X_train_input = np.array(X_train_input)
y_train = np.array(y_train)

X_test_input = np.array(X_test_input)
y_test = np.array(y_test)

X_holdout_input = np.array(X_holdout_input)
y_holdout = np.array(y_holdout)

In [13]:
print(X_train_input.shape)
print(X_test_input.shape)
print(X_holdout_input.shape)
print(y_train.shape)
print(y_test.shape)
print(y_holdout.shape)

(14289, 5, 59)
(4083, 5, 59)
(2042, 5, 59)
(14289,)
(4083,)
(2042,)


In [14]:
print(X_train_input[:10])
print(y_train[0:10])

[[[-0.09635218 -0.01836304  0.57357206 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.09635218 -0.01836304  0.57357206 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.09635218 -0.01836304  0.57357206 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.54658805  0.36974698  0.61426125 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.54658805  0.17569197  0.69563963 ... -0.14891649 -0.42154144
   -0.37139068]]

 [[-0.32147012 -0.21241805 -0.19952254 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.09635218 -0.60052807 -0.03676578 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.09635218 -0.01836304  0.12599098 ... -0.14891649 -0.42154144
   -0.37139068]
  [-0.09635218 -0.01836304 -0.24021173 ... -0.14891649 -0.42154144
   -0.37139068]
  [ 0.57900163 -0.01836304  0.28874774 ... -0.14891649 -0.42154144
   -0.37139068]]

 [[ 0.12876576 -0.21241805 -0.80986039 ...  6.715173   -0.42154144
   -0.37139068]
  [ 0.12876576 -0.21241805 -0.80986039 ...  6.715173   -0.42154144
   -0.37139068]


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

In [35]:
# # Example sequential measurement data
# # Assuming each sequence has 5 time steps and 59 features
# # X = np.random.randn(14289, 5, 59)
# X_train = torch.tensor(X_train_input, dtype=torch.float32)
# X_test = torch.tensor(X_test_input, dtype=torch.float32)
# X_holdout = torch.tensor(X_holdout_input, dtype=torch.float32)

# # Example classification labels
# # y = np.random.randint(0, 3, 14289)  # Assuming three classes
# y_train = torch.tensor(y_train, dtype=torch.long)
# y_test = torch.tensor(y_test, dtype=torch.long)
# y_holdout = torch.tensor(y_holdout, dtype=torch.long)

# # Define LSTM model for classification
# class LSTMClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(LSTMClassifier, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, num_classes)

#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])
#         return out

# # Instantiate the classification model
# input_size = 59  # Number of features in each time step
# hidden_size = 100  # Number of LSTM units
# num_layers = 2  # Number of LSTM layers
# num_classes = 3  # Number of output classes
# classification_model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

# # Train the classification model
# num_epochs = 1000
# for epoch in range(num_epochs):
#     classification_model.train()
#     optimizer.zero_grad()
#     outputs = classification_model(X_train)
#     loss = criterion(outputs, y_train)
#     loss.backward()
#     optimizer.step()
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


C:\Users\ngqin\AppData\Local\Temp\ipykernel_6852\1248966963.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\ngqin\AppData\Local\Temp\ipykernel_6852\1248966963.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.long)
C:\Users\ngqin\AppData\Local\Temp\ipykernel_6852\1248966963.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_holdout = torch.tensor(y_holdout, dtype=torch.long)


Epoch [1/1000], Loss: 1.1150
Epoch [2/1000], Loss: 1.1093
Epoch [3/1000], Loss: 1.1039
Epoch [4/1000], Loss: 1.0987
Epoch [5/1000], Loss: 1.0935
Epoch [6/1000], Loss: 1.0882
Epoch [7/1000], Loss: 1.0829
Epoch [8/1000], Loss: 1.0775
Epoch [9/1000], Loss: 1.0720
Epoch [10/1000], Loss: 1.0666
Epoch [11/1000], Loss: 1.0615
Epoch [12/1000], Loss: 1.0573
Epoch [13/1000], Loss: 1.0545
Epoch [14/1000], Loss: 1.0534
Epoch [15/1000], Loss: 1.0536
Epoch [16/1000], Loss: 1.0534
Epoch [17/1000], Loss: 1.0519
Epoch [18/1000], Loss: 1.0493
Epoch [19/1000], Loss: 1.0463
Epoch [20/1000], Loss: 1.0437
Epoch [21/1000], Loss: 1.0417
Epoch [22/1000], Loss: 1.0402
Epoch [23/1000], Loss: 1.0391
Epoch [24/1000], Loss: 1.0382
Epoch [25/1000], Loss: 1.0371
Epoch [26/1000], Loss: 1.0360
Epoch [27/1000], Loss: 1.0347
Epoch [28/1000], Loss: 1.0332
Epoch [29/1000], Loss: 1.0316
Epoch [30/1000], Loss: 1.0300
Epoch [31/1000], Loss: 1.0285
Epoch [32/1000], Loss: 1.0270
Epoch [33/1000], Loss: 1.0256
Epoch [34/1000], Lo

In [16]:
# Example sequential measurement data
# Assuming each sequence has 5 time steps and 59 features
# X = np.random.randn(14289, 5, 59)
X_train = torch.tensor(X_train_input, dtype=torch.float32)
X_test = torch.tensor(X_test_input, dtype=torch.float32)
X_holdout = torch.tensor(X_holdout_input, dtype=torch.float32)

# Example classification labels
# y = np.random.randint(0, 3, 14289)  # Assuming three classes
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
y_holdout = torch.tensor(y_holdout, dtype=torch.long)

# Define LSTM model for classification
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.0):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Defining dropout layer with specified dropout rate
        self.dropout = nn.Dropout(dropout_rate)
        
        self.fc = nn.Linear(hidden_size, num_classes)

        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        # Applying dropout
        out = self.dropout(out)
        
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        
        return out

# Instantiate the classification model
input_size = 59  # Number of features in each time step
hidden_size = 118  # Number of LSTM units
num_layers = 3  # Number of LSTM layers
num_classes = 2  # Number of output classes
dropout_rate = 0.3  # Example dropout rate
classification_model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes, dropout_rate)
# classification_model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

# Train the classification model
num_epochs = 300
for epoch in range(num_epochs):
    # classification_model.train()
    optimizer.zero_grad()
    outputs = classification_model.forward(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/300], Loss: 0.6892
Epoch [2/300], Loss: 0.6817
Epoch [3/300], Loss: 0.6745
Epoch [4/300], Loss: 0.6670
Epoch [5/300], Loss: 0.6592
Epoch [6/300], Loss: 0.6508
Epoch [7/300], Loss: 0.6413
Epoch [8/300], Loss: 0.6304
Epoch [9/300], Loss: 0.6180
Epoch [10/300], Loss: 0.6036
Epoch [11/300], Loss: 0.5869
Epoch [12/300], Loss: 0.5683
Epoch [13/300], Loss: 0.5484
Epoch [14/300], Loss: 0.5298
Epoch [15/300], Loss: 0.5142
Epoch [16/300], Loss: 0.5031
Epoch [17/300], Loss: 0.4960
Epoch [18/300], Loss: 0.4926
Epoch [19/300], Loss: 0.4909
Epoch [20/300], Loss: 0.4902
Epoch [21/300], Loss: 0.4898
Epoch [22/300], Loss: 0.4896
Epoch [23/300], Loss: 0.4893
Epoch [24/300], Loss: 0.4891
Epoch [25/300], Loss: 0.4888
Epoch [26/300], Loss: 0.4885
Epoch [27/300], Loss: 0.4883
Epoch [28/300], Loss: 0.4879
Epoch [29/300], Loss: 0.4876
Epoch [30/300], Loss: 0.4874
Epoch [31/300], Loss: 0.4872
Epoch [32/300], Loss: 0.4871
Epoch [33/300], Loss: 0.4870
Epoch [34/300], Loss: 0.4868
Epoch [35/300], Loss: 0

In [31]:
# Binary classification with BCEloss
# Assuming each sequence has 5 time steps and 59 features

X_train = torch.tensor(X_train_input, dtype=torch.float32)
X_test = torch.tensor(X_test_input, dtype=torch.float32)
X_holdout = torch.tensor(X_holdout_input, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
y_holdout = torch.tensor(y_holdout, dtype=torch.long)

# Define LSTM model for classification
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.0):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Defining dropout layer with specified dropout rate
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()
        self.fc = nn.Linear(hidden_size, num_classes)

        #self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        # Applying dropout
        out = self.dropout(out)
        
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        
        return out

# Instantiate the classification model
input_size = 59  # Number of features in each time step
hidden_size = 118  # Number of LSTM units
num_layers = 3  # Number of LSTM layers
num_classes = 1  # Number of output classes
dropout_rate = 0.3  # Example dropout rate
classification_model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes, dropout_rate)
# classification_model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

# Train the classification model
num_epochs = 300
for epoch in range(num_epochs):
    # classification_model.train()
    optimizer.zero_grad()
    outputs = classification_model.forward(X_train).squeeze(dim=1)
    y_train = y_train.float()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

C:\Users\ngqin\AppData\Local\Temp\ipykernel_12092\4032854741.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
C:\Users\ngqin\AppData\Local\Temp\ipykernel_12092\4032854741.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test, dtype=torch.long)
C:\Users\ngqin\AppData\Local\Temp\ipykernel_12092\4032854741.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_holdout = torch.tensor(y_holdout, dtype=torch.long)


Epoch [1/300], Loss: 0.6897
Epoch [2/300], Loss: 0.6777
Epoch [3/300], Loss: 0.6656
Epoch [4/300], Loss: 0.6531
Epoch [5/300], Loss: 0.6396
Epoch [6/300], Loss: 0.6246
Epoch [7/300], Loss: 0.6073
Epoch [8/300], Loss: 0.5879
Epoch [9/300], Loss: 0.5654
Epoch [10/300], Loss: 0.5396
Epoch [11/300], Loss: 0.5122
Epoch [12/300], Loss: 0.4845
Epoch [13/300], Loss: 0.4629
Epoch [14/300], Loss: 0.4536
Epoch [15/300], Loss: 0.4608
Epoch [16/300], Loss: 0.4737
Epoch [17/300], Loss: 0.4787
Epoch [18/300], Loss: 0.4762
Epoch [19/300], Loss: 0.4682
Epoch [20/300], Loss: 0.4604
Epoch [21/300], Loss: 0.4520
Epoch [22/300], Loss: 0.4476
Epoch [23/300], Loss: 0.4461
Epoch [24/300], Loss: 0.4459
Epoch [25/300], Loss: 0.4470
Epoch [26/300], Loss: 0.4499
Epoch [27/300], Loss: 0.4513
Epoch [28/300], Loss: 0.4505
Epoch [29/300], Loss: 0.4503
Epoch [30/300], Loss: 0.4492
Epoch [31/300], Loss: 0.4478
Epoch [32/300], Loss: 0.4457
Epoch [33/300], Loss: 0.4438
Epoch [34/300], Loss: 0.4417
Epoch [35/300], Loss: 0

In [32]:
from sklearn.metrics import classification_report

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_test)
    # _, predicted = torch.max(outputs, 1)
    predicted = torch.argmax(outputs, dim=1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_test.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      3321
           1       0.00      0.00      0.00       762

    accuracy                           0.81      4083
   macro avg       0.41      0.50      0.45      4083
weighted avg       0.66      0.81      0.73      4083



C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [33]:
from sklearn.metrics import classification_report

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_holdout)
    # _, predicted = torch.max(outputs, 1)
    predicted = torch.argmax(outputs, dim=1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_holdout.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      1684
           1       0.00      0.00      0.00       358

    accuracy                           0.82      2042
   macro avg       0.41      0.50      0.45      2042
weighted avg       0.68      0.82      0.75      2042



C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ngqin\anaconda3\envs\python3.10\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [34]:
from sklearn.metrics import confusion_matrix

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_test)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Compute confusion matrix
cm = confusion_matrix(y_test.numpy(), predicted)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[3321    0]
 [ 762    0]]


In [20]:
# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_train)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_train.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92     11753
           1       0.88      0.17      0.29      2536

    accuracy                           0.85     14289
   macro avg       0.86      0.58      0.60     14289
weighted avg       0.85      0.85      0.80     14289



In [96]:
from sklearn.metrics import confusion_matrix

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_train)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Compute confusion matrix
cm = confusion_matrix(y_train.numpy(), predicted)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[4664  674  910]
 [ 770 2176 1252]
 [  76  319 3448]]


In [25]:
# Define RNN model for classification
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.0):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        # Defining dropout layer with specified dropout rate
        self.dropout = nn.Dropout(dropout_rate)
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        
        # Applying dropout
        out = self.dropout(out)
        
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out

# Instantiate the classification model
input_size = 59  # Number of features in each time step
hidden_size = 118  # Number of RNN units
num_layers = 2  # Number of RNN layers
num_classes = 2  # Number of output classes
dropout_rate = 0.4  # Example dropout rate
classification_model = RNNClassifier(input_size, hidden_size, num_layers, num_classes, dropout_rate)
# classification_model = RNNClassifier(input_size, hidden_size, num_layers, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

# Train the classification model
num_epochs = 300
for epoch in range(num_epochs):
    # classification_model.train()
    optimizer.zero_grad()
    outputs = classification_model.forward(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/300], Loss: 0.6933
Epoch [2/300], Loss: 0.6580
Epoch [3/300], Loss: 0.6255
Epoch [4/300], Loss: 0.5947
Epoch [5/300], Loss: 0.5659
Epoch [6/300], Loss: 0.5407
Epoch [7/300], Loss: 0.5200
Epoch [8/300], Loss: 0.5061
Epoch [9/300], Loss: 0.4979
Epoch [10/300], Loss: 0.4938
Epoch [11/300], Loss: 0.4919
Epoch [12/300], Loss: 0.4910
Epoch [13/300], Loss: 0.4905
Epoch [14/300], Loss: 0.4901
Epoch [15/300], Loss: 0.4897
Epoch [16/300], Loss: 0.4896
Epoch [17/300], Loss: 0.4897
Epoch [18/300], Loss: 0.4894
Epoch [19/300], Loss: 0.4889
Epoch [20/300], Loss: 0.4886
Epoch [21/300], Loss: 0.4883
Epoch [22/300], Loss: 0.4881
Epoch [23/300], Loss: 0.4877
Epoch [24/300], Loss: 0.4877
Epoch [25/300], Loss: 0.4876
Epoch [26/300], Loss: 0.4873
Epoch [27/300], Loss: 0.4872
Epoch [28/300], Loss: 0.4869
Epoch [29/300], Loss: 0.4866
Epoch [30/300], Loss: 0.4866
Epoch [31/300], Loss: 0.4862
Epoch [32/300], Loss: 0.4863
Epoch [33/300], Loss: 0.4860
Epoch [34/300], Loss: 0.4858
Epoch [35/300], Loss: 0

In [26]:
from sklearn.metrics import classification_report

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_test)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_test.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.82      0.99      0.89      3321
           1       0.41      0.03      0.06       762

    accuracy                           0.81      4083
   macro avg       0.61      0.51      0.48      4083
weighted avg       0.74      0.81      0.74      4083



In [27]:
from sklearn.metrics import classification_report

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_holdout)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_holdout.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.83      0.99      0.90      1684
           1       0.40      0.03      0.06       358

    accuracy                           0.82      2042
   macro avg       0.61      0.51      0.48      2042
weighted avg       0.75      0.82      0.75      2042



In [28]:
from sklearn.metrics import confusion_matrix

# Evaluate the classification model
classification_model.eval()
with torch.no_grad():
    outputs = classification_model(X_test)
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Compute confusion matrix
cm = confusion_matrix(y_test.numpy(), predicted)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[3286   35]
 [ 738   24]]


In [133]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
input_size = 59
hidden_size = 128
num_layers = 1
num_classes = 3
learning_rate = 0.001
num_epochs = 10
batch_size = 64

# Initialize model, loss function, and optimizer
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Convert data to torch tensors
X_train_tensor = torch.tensor(time_input, dtype=torch.float32)
y_train_tensor = torch.tensor(labels, dtype=torch.long)

# Train the model
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Test the model
with torch.no_grad():
    outputs = model(torch.tensor(X_test))
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == torch.argmax(y_test, dim=1)).sum().item() / y_test.size(0)
    print(f'Test Accuracy: {accuracy}')

Epoch [1/10], Loss: 1.2430170774459839
Epoch [2/10], Loss: 1.2445777654647827
Epoch [3/10], Loss: 1.2082958221435547
Epoch [4/10], Loss: 1.2565702199935913
Epoch [5/10], Loss: 1.2094172239303589
Epoch [6/10], Loss: 1.1871225833892822
Epoch [7/10], Loss: 1.215932846069336
Epoch [8/10], Loss: 1.208574891090393
Epoch [9/10], Loss: 1.1900966167449951
Epoch [10/10], Loss: 1.2072184085845947


NameError: name 'X_test' is not defined

In [21]:
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, input_channels, num_classes, dropout_rate=0.0):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = F.relu(F.max_pool1d(self.conv1(x), 2))
        x = F.relu(F.max_pool1d(self.conv2(x), 2))
        x = x.view(x.size(0), -1)  # Flatten the output for fully connected layers
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.softmax(x)
        return x

# Instantiate the CNN-based classification model
input_channels = 59  # Number of input channels/features
num_classes = 2  # Number of output classes
dropout_rate = 0.4  # Example dropout rate
cnn_classification_model = CNNClassifier(input_channels, num_classes, dropout_rate)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_classification_model.parameters(), lr=0.001)

# Train the CNN-based classification model
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = cnn_classification_model.forward(X_train.transpose(1, 2))  # Transpose input for CNN
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 0.6948
Epoch [2/1000], Loss: 0.6923
Epoch [3/1000], Loss: 0.6898
Epoch [4/1000], Loss: 0.6856
Epoch [5/1000], Loss: 0.6798
Epoch [6/1000], Loss: 0.6717
Epoch [7/1000], Loss: 0.6626
Epoch [8/1000], Loss: 0.6523
Epoch [9/1000], Loss: 0.6410
Epoch [10/1000], Loss: 0.6292
Epoch [11/1000], Loss: 0.6176
Epoch [12/1000], Loss: 0.6043
Epoch [13/1000], Loss: 0.5914
Epoch [14/1000], Loss: 0.5783
Epoch [15/1000], Loss: 0.5662
Epoch [16/1000], Loss: 0.5540
Epoch [17/1000], Loss: 0.5414
Epoch [18/1000], Loss: 0.5312
Epoch [19/1000], Loss: 0.5224
Epoch [20/1000], Loss: 0.5151
Epoch [21/1000], Loss: 0.5090
Epoch [22/1000], Loss: 0.5037
Epoch [23/1000], Loss: 0.5001
Epoch [24/1000], Loss: 0.4974
Epoch [25/1000], Loss: 0.4953
Epoch [26/1000], Loss: 0.4938
Epoch [27/1000], Loss: 0.4927
Epoch [28/1000], Loss: 0.4922
Epoch [29/1000], Loss: 0.4916
Epoch [30/1000], Loss: 0.4913
Epoch [31/1000], Loss: 0.4909
Epoch [32/1000], Loss: 0.4910
Epoch [33/1000], Loss: 0.4908
Epoch [34/1000], Lo

In [22]:
from sklearn.metrics import classification_report

# Evaluate the classification model
cnn_classification_model.eval()
with torch.no_grad():
    outputs = cnn_classification_model(X_test.transpose(1, 2))
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_test.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.82      0.98      0.89      3321
           1       0.47      0.07      0.12       762

    accuracy                           0.81      4083
   macro avg       0.64      0.53      0.51      4083
weighted avg       0.76      0.81      0.75      4083



In [24]:
from sklearn.metrics import classification_report

# Evaluate the classification model
cnn_classification_model.eval()
with torch.no_grad():
    outputs = cnn_classification_model(X_holdout.transpose(1, 2))
    _, predicted = torch.max(outputs, 1)

# Convert predicted tensor to numpy array
predicted = predicted.numpy()

# Print classification report
print(classification_report(y_holdout.numpy(), predicted))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90      1684
           1       0.46      0.07      0.13       358

    accuracy                           0.82      2042
   macro avg       0.65      0.53      0.51      2042
weighted avg       0.77      0.82      0.77      2042



In [1]:
import torch
import torch.nn as nn
import numpy as np

# Example sequential measurement data
# Assuming each sequence has 10 time steps and 3 features
X = np.random.randn(100, 10, 3)
X = torch.tensor(X, dtype=torch.float32)

# Example regression labels
y = np.random.randn(100, 1)
y = torch.tensor(y, dtype=torch.float32)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Using only the last time step's output
        return output

# Instantiate the model
input_size = 3  # Number of features in each time step
hidden_size = 50  # Number of LSTM units
output_size = 1  # Output size (single regression value)
model = LSTMModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X)
    loss = criterion(outputs, y)
    print("Mean Squared Error on Test Data:", loss.item())

# Make predictions
predictions = outputs.numpy()

Epoch [1/50], Loss: 1.2754
Epoch [2/50], Loss: 1.2713
Epoch [3/50], Loss: 1.2676
Epoch [4/50], Loss: 1.2642
Epoch [5/50], Loss: 1.2611
Epoch [6/50], Loss: 1.2582
Epoch [7/50], Loss: 1.2556
Epoch [8/50], Loss: 1.2532
Epoch [9/50], Loss: 1.2510
Epoch [10/50], Loss: 1.2489
Epoch [11/50], Loss: 1.2470
Epoch [12/50], Loss: 1.2452
Epoch [13/50], Loss: 1.2433
Epoch [14/50], Loss: 1.2415
Epoch [15/50], Loss: 1.2396
Epoch [16/50], Loss: 1.2375
Epoch [17/50], Loss: 1.2354
Epoch [18/50], Loss: 1.2332
Epoch [19/50], Loss: 1.2309
Epoch [20/50], Loss: 1.2284
Epoch [21/50], Loss: 1.2259
Epoch [22/50], Loss: 1.2234
Epoch [23/50], Loss: 1.2207
Epoch [24/50], Loss: 1.2180
Epoch [25/50], Loss: 1.2152
Epoch [26/50], Loss: 1.2123
Epoch [27/50], Loss: 1.2093
Epoch [28/50], Loss: 1.2063
Epoch [29/50], Loss: 1.2031
Epoch [30/50], Loss: 1.1998
Epoch [31/50], Loss: 1.1964
Epoch [32/50], Loss: 1.1929
Epoch [33/50], Loss: 1.1893
Epoch [34/50], Loss: 1.1858
Epoch [35/50], Loss: 1.1822
Epoch [36/50], Loss: 1.1787
E

In [2]:
predictions

array([[ 0.53983414],
       [ 0.02595963],
       [ 0.4001707 ],
       [ 0.35539943],
       [ 0.69263095],
       [ 0.458923  ],
       [ 0.45358214],
       [ 0.6972333 ],
       [ 0.01918528],
       [-0.64251935],
       [-0.04402913],
       [-0.0785234 ],
       [-0.08128691],
       [ 0.5547164 ],
       [ 0.09887768],
       [ 0.4697313 ],
       [ 0.26392022],
       [ 0.0141659 ],
       [-0.01234652],
       [ 0.3830874 ],
       [-0.3867787 ],
       [ 0.52545273],
       [ 0.43124145],
       [ 0.0906188 ],
       [-0.23529367],
       [-0.1234112 ],
       [-0.27464283],
       [ 0.10087299],
       [ 0.4842308 ],
       [ 0.1294104 ],
       [-0.07540329],
       [ 0.20075747],
       [-0.47554547],
       [ 0.11421708],
       [ 0.6245119 ],
       [ 0.5454791 ],
       [-0.3282947 ],
       [ 0.05051087],
       [ 0.39321065],
       [-0.03295558],
       [ 0.33816934],
       [ 0.47598463],
       [ 0.09506223],
       [-0.00884948],
       [ 0.41988903],
       [-0

In [3]:
len(predictions)

100

In [ ]:
h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)